In [1]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [2]:
from strategy.primitives import POOLS, Pool
pool_num = 1
pool = Pool(
    tokenA=POOLS[pool_num]['token0'],
    tokenB=POOLS[pool_num]['token1'],
    fee=POOLS[pool_num]['fee']
)

In [ ]:
from strategy.data import DownloaderRawDataUniV3
# downloads mint, burn, swap for the pool, this needs to be done once the files are added to ../data/
# ~5min
DownloaderRawDataUniV3().load_events(pool_num)

get burn
saved to burn_WBTC_WETH_3000.csv
get mint
saved to mint_WBTC_WETH_3000.csv
get swap


In [ ]:
from strategy.data import RawDataUniV3
data = RawDataUniV3(pool).load_from_folder()

In [ ]:
from strategy.strategies import AbstractStrategy
from strategy.uniswap_utils import UniswapLiquidityAligner
from strategy.positions import UniV3Position

class UniV3Passive(AbstractStrategy):
    """
    ``UniV3Passive`` is the passive strategy on UniswapV3 without rebalances.
        i.e. mint interval and wait.
        lower_price: Lower bound of the interval
        upper_price: Upper bound of the interval
        rebalance_cost: Rebalancing cost, expressed in currency
        pool: UniswapV3 Pool instance
        name: Unique name for the instance
    """
    def __init__(self,
                 lower_price: float,
                 upper_price: float,
                 pool: Pool,
                 rebalance_cost: float,
                 name: str = None,
                 ):
        super().__init__(name)
        self.lower_price = lower_price
        self.upper_price = upper_price
        self.decimal_diff = -pool.decimals_diff
        self.fee_percent = pool.fee.percent
        self.rebalance_cost = rebalance_cost

    def rebalance(self, *args, **kwargs) -> str:
        timestamp = kwargs['timestamp']
        row = kwargs['row']
        portfolio = kwargs['portfolio']
        price_before, price = row['price_before'], row['price']

        
        if len(portfolio.positions) == 0:
            univ3_pos = self.create_uni_position(price)
            portfolio.append(univ3_pos)
            return 'mint'
        
        uni_pos = portfolio.get_position('UniV3Passive')
        uni_pos.charge_fees(price_before, price)


    def create_uni_position(self, price):
        univ3_pos = UniV3Position('UniV3Passive', self.lower_price, self.upper_price, self.fee_percent, self.rebalance_cost)
        x_uni_aligned, y_uni_aligned = univ3_pos.swap_to_optimal(x=1 / price, y=1, price=price)
        univ3_pos.deposit(x=x_uni_aligned, y=y_uni_aligned, price=price)
        return univ3_pos

In [ ]:
from strategy.backtest import Backtest

univ3_passive = UniV3Passive(
    lower_price=data.swaps['price'].min() + 1,
    upper_price=data.swaps['price'].max() - 1,
    pool=pool,
    rebalance_cost=0.,
    name='passive'
)
b = Backtest(univ3_passive)
portfolio_history, rebalance_history, uni_history = b.backtest(data.swaps)

In [ ]:
from strategy.viewers import RebalanceViewer, UniswapViewer, PotrfolioViewer

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly

rv = RebalanceViewer(rebalance_history)
uv = UniswapViewer(uni_history)
pv = PotrfolioViewer(portfolio_history, pool)

# Draw portfolio stats, like value, fees earned, apy
fig1, fig2, fig3, fig4, fig5, fig6 = pv.draw_portfolio()

# Draw Uniswap intervals
intervals_plot = uv.draw_intervals(data.swaps)

# Draw rebalances
rebalances_plot = rv.draw_rebalances(data.swaps)

# Calculate df with portfolio stats
stats = portfolio_history.calculate_stats()

In [ ]:
intervals_plot.update_layout(height=300, width=800)

In [ ]:
rebalances_plot.update_layout(height=300, width=800)

In [ ]:
fig2.update_layout(height=300, width=800)

In [ ]:
fig4.update_layout(height=300, width=800)

In [ ]:
fig6.update_layout(height=300, width=800)

In [ ]:
stats.tail(3)

In [ ]:
stats.tail(3).to_csv('stats.csv')